In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [50]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet").head(20000)
rec_system_copy = rec_system.copy()

In [51]:
arr = []
for index,row in rec_system.iterrows():
    # print(row["name_genre"])
    # break
    for genre in row["name_genre"]:
        rec_system_copy.at[index,genre]

arr        

KeyError: 'Comedy'

In [45]:
rec_system.fillna({"overview":"[]",
                   "name_genre":"[]",
                   "actors_names":"[]",
                   "director_names":"[]",
                   "tagline":"[]",
                   "vote_average":rec_system["vote_average"].mean()},inplace=True)

* Transformo la columna overview a una lista con la funcion `split`

In [19]:
rec_system["overview"] = rec_system["overview"].apply(lambda x: x.split())
rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())
rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())

In [20]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i.replace(" ",""))
    return valores

In [ ]:
rec_system["vote_average"].astype(str)

In [21]:
rec_system["name_genre"]=rec_system["name_genre"].apply(collapse)
rec_system["actors_names"]=rec_system["actors_names"].apply(collapse)
rec_system["director_names"]=rec_system["director_names"].apply(collapse)
rec_system["tagline"]=rec_system["tagline"].apply(collapse)

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
19995    5.7
19996    6.2
19997    6.0
19998    4.8
19999    5.8
Name: vote_average, Length: 20000, dtype: object

In [26]:
rec_system["tags"] = rec_system["overview"] + rec_system["name_genre"] + rec_system["actors_names"] + rec_system["director_names"] + rec_system["tagline"] 

In [27]:
rec_system["tags"] = rec_system["tags"].apply(lambda x: "".join(x))

In [28]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,vote_average,tags
0,toy story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Comedy, Animation, Family]","[LaurieMetcalf, TimAllen, WallaceShawn, JohnMo...",[JohnLasseter],[[]],7.7,"LedbyWoody,Andy'stoyslivehappilyinhisroomuntil..."
1,jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Adventure, Fantasy, Family]","[RobinWilliams, LauraBellBundy, AnnabelKershaw...",[JoeJohnston],"[Roll, the, dice, and, unleash, the, excitement!]",6.9,WhensiblingsJudyandPeterdiscoveranenchantedboa...
2,grumpier old men,"[A, family, wedding, reignites, the, ancient, ...","[Comedy, Romance]","[WalterMatthau, JackLemmon, Ann-Margret, Burge...",[HowardDeutch],"[Still, Yelling., Still, Fighting., Still, Rea...",6.5,Afamilyweddingreignitestheancientfeudbetweenne...
3,waiting to exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Comedy, Romance, Drama]","[GregoryHines, MichaelBeach, AngelaBassett, We...",[ForestWhitaker],"[Friends, are, the, people, who, let, you, be,...",6.1,"Cheatedon,mistreatedandsteppedon,thewomenareho..."
4,father of the bride part ii,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[KieranCulkin, GeorgeNewbern, MartinShort, BDW...",[CharlesShyer],"[Just, When, His, World, Is, Back, To, Normal....",5.7,JustwhenGeorgeBankshasrecoveredfromhisdaughter...
...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,"[So, goes, to, the, U.S., to, open, a, martial...","[Western, Action]","[T.J.Storm, JohnnyKoo, RyonMarshall, PatrickLu...","[LauKar-Wing, SammoHung]",[[]],5.7,SogoestotheU.S.toopenamartialartsschool.Around...
19996,contract killer,"[Financially, troubled,, a, newbie, hitman, re...","[Comedy, Action]","[MasakoEnokimaru, MitsuruTokashiki, FukuiAkihi...",[WeiTung],[[]],6.2,"Financiallytroubled,anewbiehitmanreluctantlyta..."
19997,the executor,"[Chow, Yun-Fat, and, Danny, Lee, star, in, thi...","[Crime, Action, Drama]","[LilyLiLi-Li, DannyLeeSau-Yin, WongChung, NgMa...","[ChikNgai-Hung, Yi-HsiungChi, ChuanChen]",[[]],6.0,ChowYun-FatandDannyLeestarinthispoliceactionfi...
19998,never say... never!,"[Paris,, 1830., Valentin, loses, himself, in, ...",[Comedy],"[JeanDujardin, LorellaCravotta, JacquesHerlin,...",[EricCivanyan],[[]],4.8,"Paris,1830.Valentinloseshimselfinalcohol,gambl..."


In [29]:
# max_features: maxima cantidad de palabras a vectorizar
# cv = CountVectorizer(max_features=5000, stop_words="english")
tv = TfidfVectorizer(max_features=5000, stop_words="english")

In [30]:
vector = tv.fit_transform(rec_system["tags"]).toarray()

In [31]:
tv.vocabulary_ # Chequeo palabras que se repitieron


{'andy': 477,
 'sheart': 3793,
 'alan': 240,
 'butshe': 1010,
 'goodman': 1839,
 'bernie': 844,
 'swedding': 4128,
 'spregnant': 3960,
 'swife': 4142,
 'nina': 2908,
 'butthat': 1013,
 'jr': 2450,
 'smith': 3891,
 'send': 3658,
 'oneoftheworld': 3019,
 'awashingtonlobbyist': 730,
 'alaska': 244,
 'allhero': 279,
 'anall': 349,
 'unfortunately': 4785,
 'sonny': 3931,
 'bones': 922,
 'butit': 1000,
 'renext': 3335,
 'greed': 1852,
 'agetale': 193,
 'secrets': 3635,
 'yearslater': 4985,
 'smistress': 3890,
 'chip': 1146,
 'jacquesannaud': 2387,
 'year': 4980,
 'sdad': 3614,
 'roger': 3418,
 'clothes': 1212,
 'popularity': 3216,
 'asouth': 647,
 'shakespeare': 3773,
 'duringthe1960s': 1459,
 'eventually': 1548,
 'romancedramarobertdowneyjr': 3436,
 'tosaveearth': 4672,
 'soon': 3933,
 'calif': 1048,
 'smind': 3887,
 'johnsmith': 2439,
 'spirited': 3951,
 'detective': 1344,
 'turned': 4750,
 'amanda': 307,
 'fiore': 1648,
 'butcan': 975,
 'finally': 1643,
 'afterlosingherjob': 169,
 'boss':

In [33]:
cosine_sim = linear_kernel(vector,vector)

In [34]:
indices = pd.Series(rec_system.index, index=rec_system["title"]).drop_duplicates()

In [35]:
# similarity = cosine_similarity(vector)

In [36]:
for i in rec_system[rec_system["title"]=="toy story"]["actors_names"]:
    print(i)

['LaurieMetcalf', 'TimAllen', 'WallaceShawn', 'JohnMorris', 'AnniePotts', 'TomHanks', 'PennJillette', 'R.LeeErmey', 'DonRickles', 'ErikvonDetten', 'JimVarney', 'JohnRatzenberger', 'SarahFreeman']


In [37]:
# similarity[300]

In [38]:
def recommend(movie,cosine_sim=cosine_sim):
    movie = movie.lower()
    index = indices[movie]
    sim_scores=list(enumerate(cosine_sim[index]))
    distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
    
    for i in distances[1:10]:
        # print(i[0])
        print(rec_system.iloc[i[0]].title)
recommend("toy stORy 2")

group sex
toy story
the first $20 million is always the hardest
malice
the gang's all here
at play in the fields of the lord
life is sweet
mute witness
jumanji
